In [1]:
print("Initialize libraries")

import os

mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-5.3.0-posix-seh-rt_v4-rev0\\mingw64\\bin'

os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']

import pandas as pd
import sys
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from sklearn.cross_validation import StratifiedKFold, KFold
from sklearn.metrics import log_loss
from sklearn.cluster import DBSCAN
from sklearn import metrics as skmetrics
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from collections import Counter

from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import ensemble
from sklearn.decomposition import PCA
import gc
from scipy import sparse
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.feature_selection import SelectPercentile, f_classif, chi2, SelectKBest
from sklearn import ensemble
from sklearn.neighbors import KNeighborsClassifier


from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import train_test_split
from sklearn.metrics import log_loss

Initialize libraries


In [2]:
test = pd.read_csv("E:/HackerRank/Contest/Email/test_dataset.csv")
train1 = pd.read_csv("E:/HackerRank/Contest/Email/training_dataset.csv")


In [3]:
click_time = 'click_time'
clicked = 'clicked'
open_time = 'open_time'
unsub_time = 'unsubscribe_time'
unsub = 'unsubscribed'

predictors = [x for x in train1.columns if x not in [click_time,clicked,open_time,unsub_time,unsub]]
train = train1[predictors]

In [4]:
df = train
df.apply(lambda x: sum(x.isnull()))

user_id                                    0
mail_id                                    0
mail_category                            425
mail_type                                425
sent_time                                  0
last_online                              577
hacker_created_at                          0
hacker_timezone                         6939
contest_login_count                        0
contest_login_count_1_days                 0
contest_login_count_30_days                0
contest_login_count_365_days               0
contest_login_count_7_days                 0
contest_participation_count                0
contest_participation_count_1_days         0
contest_participation_count_30_days        0
contest_participation_count_365_days       0
contest_participation_count_7_days         0
forum_comments_count                       0
forum_count                                0
forum_expert_count                         0
forum_questions_count                      0
hacker_con

In [5]:
df.describe()

C:\Users\Prashanth.S\AppData\Local\Continuum\Anaconda2\lib\site-packages\numpy\lib\function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,sent_time,last_online,hacker_created_at,hacker_timezone,contest_login_count,contest_login_count_1_days,contest_login_count_30_days,contest_login_count_365_days,contest_login_count_7_days,contest_participation_count,...,submissions_count_contest,submissions_count_contest_1_days,submissions_count_contest_30_days,submissions_count_contest_365_days,submissions_count_contest_7_days,submissions_count_master,submissions_count_master_1_days,submissions_count_master_30_days,submissions_count_master_365_days,submissions_count_master_7_days
count,4.860480e+05,4.854710e+05,4.860480e+05,479109.000000,486048.000000,486048.000000,486048.000000,486048.000000,486048.000000,486048.000000,...,486048.000000,486048.000000,486048.000000,486048.000000,486048.000000,486048.000000,486048.000000,486048.000000,486048.000000,486048.000000
mean,1.460378e+09,1.459161e+09,1.439918e+09,5245.825063,3.696357,0.009674,0.524516,2.885806,0.162198,5.832628,...,17.069950,0.047257,1.598953,13.445485,0.486911,40.622459,0.154602,9.030365,35.104648,2.471916
std,2.143160e+06,3.300271e+06,2.278764e+07,17553.931062,7.689379,0.107343,0.904353,5.626671,0.443127,12.342502,...,78.252872,0.690251,7.868896,48.730845,3.246356,67.741608,1.365345,20.450653,58.196637,7.836031
min,1.455281e+09,1.376045e+09,1.336762e+09,-43200.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.458833e+09,NaN,1.429803e+09,NaN,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000,2.000000,0.000000
50%,1.460575e+09,NaN,1.448939e+09,NaN,1.000000,0.000000,0.000000,1.000000,0.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,16.000000,0.000000,1.000000,14.000000,0.000000
75%,1.462287e+09,NaN,1.457395e+09,NaN,3.000000,0.000000,1.000000,3.000000,0.000000,5.000000,...,6.000000,0.000000,0.000000,5.000000,0.000000,48.000000,0.000000,9.000000,43.000000,1.000000
max,1.463529e+09,1.463524e+09,1.463368e+09,50400.000000,123.000000,4.000000,16.000000,94.000000,8.000000,214.000000,...,4549.000000,83.000000,243.000000,1195.000000,167.000000,1310.000000,138.000000,641.000000,1310.000000,553.000000


In [6]:
from scipy.stats import mode
#df['mail_category'].fillna(mode(df['mail_category']).mode[0],inplace = True)
#df['mail_type'].fillna(mode(df['mail_type']).mode[0],inplace = True)

In [7]:
#df['last_online'].fillna(np.math.ceil(np.mean(df['last_online'])),inplace = True)
#df['hacker_timezone'].fillna(mode(df['hacker_timezone']).mode[0],inplace = True)

In [8]:
df.dtypes

user_id                                  object
mail_id                                  object
mail_category                            object
mail_type                                object
sent_time                                 int64
last_online                             float64
hacker_created_at                         int64
hacker_timezone                         float64
contest_login_count                       int64
contest_login_count_1_days                int64
contest_login_count_30_days               int64
contest_login_count_365_days              int64
contest_login_count_7_days                int64
contest_participation_count               int64
contest_participation_count_1_days        int64
contest_participation_count_30_days       int64
contest_participation_count_365_days      int64
contest_participation_count_7_days        int64
forum_comments_count                      int64
forum_count                               int64
forum_expert_count                      

In [9]:
df['opened'] = np.where(df['opened']==True,1,0)

C:\Users\Prashanth.S\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [10]:
df['sent_time'] = (df['sent_time']/86400)
#df['last_online'] = int(np.math.floor(df['last_online']))
df['last_online'] = (df['last_online']/86400)
df['hacker_created_at'] =(df['hacker_created_at']/86400)
#df['hacker_timezone'] = int(np.math.floor(df['hacker_timezone']))
df['hacker_timezone'] = (df['hacker_timezone']/3600)
df['time_diff'] = df['sent_time'] - df['last_online']
#df['time_hacker'] = df['sent_time'] - df['hacker_created_at']
df['read_count_per'] = ((df['ipn_read']).astype('float') / df['ipn_count']).astype('float')*100
df['submissions_count_contest_per'] = ((df['submissions_count_contest']).astype('float')/df['submissions_count']).astype('float')*100
df['submissions_count_contest_per_1'] = ((df['submissions_count_contest_1_days']).astype('float')/df['submissions_count_1_days']).astype('float')*100
df['submissions_count_contest_per_30'] = ((df['submissions_count_contest_30_days']).astype('float')/df['submissions_count_30_days']).astype('float')*100
df['submissions_count_contest_per_365'] = ((df['submissions_count_contest_365_days']).astype('float')/df['submissions_count_365_days']).astype('float')*100
df['submissions_count_contest_per_7'] = ((df['submissions_count_contest_7_days']).astype('float')/df['submissions_count_7_days']).astype('float')*100
df['submissions_count_master_per'] = ((df['submissions_count_master']).astype('float') / df['submissions_count']).astype('float')*100
df['submissions_count_master_per_1'] = ((df['submissions_count_master_1_days']).astype('float') / df['submissions_count_1_days']).astype('float')*100
df['submissions_count_master_per_30'] = ((df['submissions_count_master_30_days']).astype('float') / df['submissions_count_30_days']).astype('float')*100
df['submissions_count_master_per_365'] = ((df['submissions_count_master_365_days']).astype('float') / df['submissions_count_365_days']).astype('float')
df['submissions_count_master_per_7'] = ((df['submissions_count_master_7_days']).astype('float') / df['submissions_count_7_days']).astype('float')*100
df['contest_login_count_per'] = ((df['contest_login_count']).astype('float') / df['contest_participation_count']).astype('float')*100
#df['forum_comments_count_per'] = ((df['forum_comments_count']).astype('float') / df['forum_count']).astype('float')*100
#df['forum_expert_count_per'] = ((df['forum_expert_count']).astype('float') / df['forum_count']).astype('float')*100
#df['forum_questions_count_per'] = ((df['forum_questions_count']).astype('float') / df['forum_count']).astype('float')*100

C:\Users\Prashanth.S\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\Prashanth.S\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\Prashanth.S\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

In [11]:
df.head()

,user_id,mail_id,mail_category,mail_type,sent_time,last_online,hacker_created_at,hacker_timezone,contest_login_count,contest_login_count_1_days,...,submissions_count_contest_per_1,submissions_count_contest_per_30,submissions_count_contest_per_365,submissions_count_contest_per_7,submissions_count_master_per,submissions_count_master_per_1,submissions_count_master_per_30,submissions_count_master_per_365,submissions_count_master_per_7,contest_login_count_per
0,AR+tMy3H/E+Re8Id20zUIz+amJkv6KU12o+BrgIDin0=,DQ/4I+GIOz2ZoIiK0Lg0AkwnI35XotghgUK/MYc101I=,mail_category_2,mail_type_1,16938.632373,16892.595000,16580.243322,5.0,1,0,...,NaN,NaN,0.000000,NaN,100.000000,NaN,NaN,1.000000,NaN,100.000000
1,1P4AOvdzJzhDSHi7jJ3udWv4ajpKxOn4T/rCLv4PrXU=,BL3z4RtiyfIDydaRYWX2ZXL6IX10QH1yG5ak1s/8Lls=,mail_category_1,mail_type_1,16913.861574,16912.157025,16576.207072,-7.0,3,0,...,NaN,6.521739,16.161616,0.0,83.838384,NaN,93.478261,0.838384,100.0,100.000000
2,OEfFUcsTAGInCfsHuLZuIgdSNtuNsg8EdfN98VUZVTs=,BL3z4RtiyfIDydaRYWX2ZXL6IX10QH1yG5ak1s/8Lls=,mail_category_1,mail_type_1,16938.653229,16937.628148,16586.179826,5.0,3,0,...,NaN,0.000000,0.000000,NaN,100.000000,NaN,100.000000,1.000000,NaN,100.000000
3,1P4AOvdzJzhDSHi7jJ3udWv4ajpKxOn4T/rCLv4PrXU=,EHNBRbi6i9KO6cMHsuDPFjZVp2cY3RH+BiOKwPwzLQs=,mail_category_1,mail_type_1,16934.988229,16930.184745,16576.207072,-7.0,3,0,...,NaN,0.000000,15.841584,NaN,84.158416,NaN,100.000000,0.841584,NaN,100.000000
4,CYRcuV0cR0algMZJ1N6+3uKcqi8iu+6tJNzmBbmgN7o=,K0y/NW59TJkYc5y0HUwDeAXrewYT0JQlkcozz0s2V5Q=,mail_category_4,mail_type_1,16919.618275,16912.597477,16585.625671,5.0,5,0,...,NaN,100.000000,28.333333,NaN,71.666667,NaN,0.000000,0.716667,NaN,38.461538


In [11]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [12]:
dt = pd.DataFrame()
dt = pysqldf('select user_id,count(user_id) as mail_sent_count,sum(opened) as opened_count from df group by user_id')

In [13]:
dt.head()

,user_id,mail_sent_count,opened_count
0,++++AlrUVbE5PINY7DbtKaSSWKVBd1irRDUEQazAlB0=,30,5
1,++E9b50RAFVUBx06HLYizCphGFSdewHiJ6O7R8dkb0w=,8,2
2,++VKdjPb2qYO+KV9T2r+wcFpg+Fz2B0ZlxO2KtCUQ24=,18,0
3,++Xam77eQwNSqOVgGAJufeNYduq3iqIRyvSuQATtoss=,35,14
4,++Zro4xLyeHDkcH0IrRRbpKek4twNACxWd40pdCM/rc=,31,6


In [14]:
df = pd.merge(df,dt,on='user_id',how='inner')

In [15]:
df.head()

,user_id,mail_id,mail_category,mail_type,sent_time,last_online,hacker_created_at,hacker_timezone,contest_login_count,contest_login_count_1_days,...,submissions_count_contest_per_365,submissions_count_contest_per_7,submissions_count_master_per,submissions_count_master_per_1,submissions_count_master_per_30,submissions_count_master_per_365,submissions_count_master_per_7,contest_login_count_per,mail_sent_count,opened_count
0,AR+tMy3H/E+Re8Id20zUIz+amJkv6KU12o+BrgIDin0=,DQ/4I+GIOz2ZoIiK0Lg0AkwnI35XotghgUK/MYc101I=,mail_category_2,mail_type_1,16938.632373,16892.595,16580.243322,5.0,1,0,...,0.0,NaN,100.0,NaN,NaN,1.0,NaN,100.0,13,12
1,AR+tMy3H/E+Re8Id20zUIz+amJkv6KU12o+BrgIDin0=,r6eVJNYsfjGvlgEw0Y6QQYh4Vvd+EUA9e0blOUEo1k0=,mail_category_10,mail_type_1,16923.625428,16892.595,16580.243322,5.0,1,0,...,0.0,NaN,100.0,NaN,NaN,1.0,NaN,100.0,13,12
2,AR+tMy3H/E+Re8Id20zUIz+amJkv6KU12o+BrgIDin0=,aLbZ3A6W9LCI8mEzMAxrgffBFns0U/V9pSVVx/yNwn4=,mail_category_15,mail_type_3,16924.154780,16892.595,16580.243322,5.0,1,0,...,0.0,NaN,100.0,NaN,NaN,1.0,NaN,100.0,13,12
3,AR+tMy3H/E+Re8Id20zUIz+amJkv6KU12o+BrgIDin0=,UGATDXARg7jMEInKH7oXgty2nwxnwD2l0OW/8Nsa0MI=,mail_category_6,mail_type_1,16915.628762,16892.595,16580.243322,5.0,1,0,...,0.0,NaN,100.0,NaN,NaN,1.0,NaN,100.0,13,12
4,AR+tMy3H/E+Re8Id20zUIz+amJkv6KU12o+BrgIDin0=,Sj580DqbWjGNy1twh+Wyg0k/Rdv4i6CdkzWu2Vk8Oy8=,mail_category_9,mail_type_1,16930.649664,16892.595,16580.243322,5.0,1,0,...,0.0,NaN,100.0,NaN,NaN,1.0,NaN,100.0,13,12


In [16]:
df['opened_ratio'] = df['opened_count'] / df['mail_sent_count']


In [17]:
df.apply(lambda x: sum(x.isnull()))

user_id                                      0
mail_id                                      0
mail_category                              425
mail_type                                  425
sent_time                                    0
last_online                                577
hacker_created_at                            0
hacker_timezone                           6939
contest_login_count                          0
contest_login_count_1_days                   0
contest_login_count_30_days                  0
contest_login_count_365_days                 0
contest_login_count_7_days                   0
contest_participation_count                  0
contest_participation_count_1_days           0
contest_participation_count_30_days          0
contest_participation_count_365_days         0
contest_participation_count_7_days           0
forum_comments_count                         0
forum_count                                  0
forum_expert_count                           0
forum_questio

In [18]:
var_mod = ['hacker_timezone']
for i in var_mod:
    dummies = pd.get_dummies(df[i],prefix=i)
    df.drop(i,axis=1,inplace=True)
    df = df.join(dummies)

In [184]:
#df =df.sort(['user_id'], ascending=[True]) 

In [19]:
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV 

In [20]:
target = 'opened'
IDcol = 'user_id'
IDcol1 = 'mail_id'
mailtype1 = 'mail_type'
mailcat18 = 'mail_category'

predictors = [x for x in df.columns if x not in [target,IDcol,IDcol1,mailtype1,mailcat18]]
Y = df['opened']
X_train, X_val, y_train, y_val = train_test_split(
    df[predictors],Y, train_size=0.90, random_state=10)

In [21]:
dtrain = xgb.DMatrix(X_train,y_train)
dvalid = xgb.DMatrix(X_val, y_val)

params = {
    "objective": "binary:logistic",
    "booster": "gbtree",
    "max_depth":6,
    "eval_metric": "error",
    "eta": 0.1,
    "silent": 1,
    'colsample':0.9,
    'subsample':0.9
}

In [22]:
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
gbm = xgb.train(params, dtrain,150, evals=watchlist,
                early_stopping_rounds=10, verbose_eval=True)

[0]	train-error:0.178368	eval-error:0.176628
Multiple eval metrics have been passed: 'eval-error' will be used for early stopping.

Will train until eval-error hasn't improved in 10 rounds.
[1]	train-error:0.177717	eval-error:0.176937
[2]	train-error:0.177321	eval-error:0.17706
[3]	train-error:0.177001	eval-error:0.176155
[4]	train-error:0.176599	eval-error:0.176052
[5]	train-error:0.176592	eval-error:0.176155
[6]	train-error:0.176167	eval-error:0.176196
[7]	train-error:0.176313	eval-error:0.175826
[8]	train-error:0.175991	eval-error:0.175558
[9]	train-error:0.176025	eval-error:0.175702
[10]	train-error:0.175995	eval-error:0.175537
[11]	train-error:0.175968	eval-error:0.175682
[12]	train-error:0.17587	eval-error:0.175537
[13]	train-error:0.175918	eval-error:0.175085
[14]	train-error:0.175831	eval-error:0.175147
[15]	train-error:0.175883	eval-error:0.175332
[16]	train-error:0.175808	eval-error:0.175044
[17]	train-error:0.175778	eval-error:0.174961
[18]	train-error:0.175767	eval-error:0.

In [23]:
dtrain_predprob = gbm.predict(dvalid)

In [24]:
dtrain_predprob = np.where(dtrain_predprob>=0.35,1,0)

In [25]:
pd.crosstab(y_val, dtrain_predprob)

col_0,0,1
opened,,
0,26282,6205
1,3064,13054


In [26]:
%pylab inline
feat_imp = pd.Series(gbm.get_fscore()).sort_values(ascending=False)
feat_imp

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


sent_time                               605
opened_ratio                            575
time_diff                               353
last_online                             155
opened_count                             86
mail_sent_count                          76
hacker_created_at                        66
ipn_count_7_days                         63
ipn_count_30_days                        51
contest_login_count_7_days               42
ipn_count                                40
submissions_count_7_days                 39
submissions_count_master_7_days          38
contest_participation_count_7_days       34
submissions_count_365_days               33
contest_login_count_per                  30
submissions_count                        29
submissions_count_contest_per            28
submissions_count_contest_per_7          28
submissions_count_master_365_days        26
submissions_count_1_days                 25
read_count_per                           23
submissions_count_30_days       

In [27]:
test = pd.read_csv("E:/HackerRank/Contest/Email/test_dataset.csv")
train = pd.read_csv("E:/HackerRank/Contest/Email/train.csv")
#test['mail_category'].fillna(mode(train['mail_category']).mode[0],inplace = True)
#test['mail_type'].fillna(mode(train['mail_type']).mode[0],inplace = True)

In [194]:
#test['last_online'].fillna(np.math.ceil(np.mean(train['last_online'])),inplace = True)
#test['hacker_timezone'].fillna(mode(train['hacker_timezone']).mode[0],inplace = True)

In [28]:
test['sent_time'] = (test['sent_time']/86400)
#test['last_online'] = int(np.math.floor(test['last_online']))
test['last_online'] = (test['last_online']/86400)
test['hacker_created_at'] =(test['hacker_created_at']/86400)
#test['hacker_timezone'] = int(np.math.floor(test['hacker_timezone']))
test['hacker_timezone'] = (test['hacker_timezone']/3600)
test['time_diff'] = test['sent_time'] - test['last_online']
#test['time_hacker'] = test['sent_time'] - test['hacker_created_at']
test['read_count_per'] = ((test['ipn_read']).astype('float') / test['ipn_count']).astype('float')*100
test['submissions_count_contest_per'] = ((test['submissions_count_contest']).astype('float')/test['submissions_count']).astype('float')*100
test['submissions_count_contest_per_1'] = ((test['submissions_count_contest_1_days']).astype('float')/test['submissions_count_1_days']).astype('float')*100
test['submissions_count_contest_per_30'] = ((test['submissions_count_contest_30_days']).astype('float')/test['submissions_count_30_days']).astype('float')*100
test['submissions_count_contest_per_365'] = ((test['submissions_count_contest_365_days']).astype('float')/test['submissions_count_365_days']).astype('float')*100
test['submissions_count_contest_per_7'] = ((test['submissions_count_contest_7_days']).astype('float')/test['submissions_count_7_days']).astype('float')*100
test['submissions_count_master_per'] = ((test['submissions_count_master']).astype('float') / test['submissions_count']).astype('float')*100
test['submissions_count_master_per_1'] = ((test['submissions_count_master_1_days']).astype('float') / test['submissions_count_1_days']).astype('float')*100
test['submissions_count_master_per_30'] = ((test['submissions_count_master_30_days']).astype('float') / test['submissions_count_30_days']).astype('float')*100
test['submissions_count_master_per_365'] = ((test['submissions_count_master_365_days']).astype('float') / test['submissions_count_365_days']).astype('float')
test['submissions_count_master_per_7'] = ((test['submissions_count_master_7_days']).astype('float') / test['submissions_count_7_days']).astype('float')*100
test['contest_login_count_per'] = ((test['contest_login_count']).astype('float') / test['contest_participation_count']).astype('float')*100
#test['forum_comments_count_per'] = ((test['forum_comments_count']).astype('float') / test['forum_count']).astype('float')*100
#test['forum_expert_count_per'] = ((test['forum_expert_count']).astype('float') / test['forum_count']).astype('float')*100
#test['forum_questions_count_per'] = ((test['forum_questions_count']).astype('float') / test['forum_count']).astype('float')*100

In [29]:
dt['opened_ratio'] = dt['opened_count'] / dt['mail_sent_count']

In [30]:
test = pd.merge(test,dt,on='user_id',how='left')

In [31]:
test.head()

,user_id,mail_id,mail_category,mail_type,sent_time,last_online,hacker_created_at,hacker_timezone,contest_login_count,contest_login_count_1_days,...,submissions_count_contest_per_7,submissions_count_master_per,submissions_count_master_per_1,submissions_count_master_per_30,submissions_count_master_per_365,submissions_count_master_per_7,contest_login_count_per,mail_sent_count,opened_count,opened_ratio
0,3EwevzgIuZX9hq2zhf1Mz2C/xOd5Rf+B793bBOTSkRA=,BL3z4RtiyfIDydaRYWX2ZXL6IX10QH1yG5ak1s/8Lls=,mail_category_1,mail_type_1,16987.366030,16986.344225,16587.331123,5.0,1,0,...,NaN,100.000000,NaN,NaN,1.000000,NaN,100.0,32.0,1.0,0.03125
1,blQUjF8SGLYtv1K236MnflXw0X6ie8QJHZy0SDW5Llg=,EHNBRbi6i9KO6cMHsuDPFjZVp2cY3RH+BiOKwPwzLQs=,mail_category_1,mail_type_1,16974.194907,16973.177801,16594.146551,5.0,3,0,...,NaN,80.000000,NaN,NaN,0.903226,NaN,75.0,31.0,0.0,0.00000
2,OEfFUcsTAGInCfsHuLZuIgdSNtuNsg8EdfN98VUZVTs=,UqjHLTfci2jErniqKM2JHBn8DurSys3+UlZOpSa32t4=,mail_category_3,mail_type_1,16940.646840,16937.628148,16586.179826,5.0,3,0,...,NaN,100.000000,NaN,100.0,1.000000,NaN,100.0,7.0,0.0,0.00000
3,4L/sXU2l7/1DyWK5BdEkmex0yBiyHaAW9yGPHMtrDOw=,BL3z4RtiyfIDydaRYWX2ZXL6IX10QH1yG5ak1s/8Lls=,mail_category_1,mail_type_1,16987.491019,16986.485498,16575.336308,5.0,2,0,...,NaN,2.380952,NaN,NaN,0.023810,NaN,100.0,20.0,1.0,0.05000
4,0OfcrqLw8m4qYC0CLqX0DvmzBV/OQZNa9GNXQKHZdK4=,K0y/NW59TJkYc5y0HUwDeAXrewYT0JQlkcozz0s2V5Q=,mail_category_4,mail_type_1,16987.537616,16980.509213,16574.215150,5.0,1,0,...,NaN,100.000000,NaN,100.0,1.000000,NaN,100.0,4.0,3.0,0.75000


In [32]:
var_mod = ['hacker_timezone']
for i in var_mod:
    dummies = pd.get_dummies(test[i],prefix=i)
    test.drop(i,axis=1,inplace=True)
    test = test.join(dummies)

In [33]:
test['out'] = gbm.predict(xgb.DMatrix(test[predictors]))

In [34]:
test['out1'] = np.where(test['out']>=0.35,1,0)

In [35]:
pd.Series.value_counts(test['out1'])

0    146950
1     60474
Name: out1, dtype: int64

In [36]:
dtest = pd.DataFrame()
dtest['val'] = test['out1']

In [37]:
dtest.to_csv('predictions_xgb.csv',header=False,index=False)